In [ ]:
!pip install dash

In [18]:
import requests
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# Create a Dash application
app = dash.Dash(__name__)

# Define the layout of the dashboard
app.layout = html.Div([
    html.H1("Flight Information Dashboard"),
    html.Label('Enter Flight Number:'),
    dcc.Input(id='flight-input', type='text', value='DL993'),
    html.Button('Get Flight Information', id='get-flight-button', n_clicks=0),
    html.Div(id='flight-output')
])

# Define a function to fetch flight information for a specific flight number
def get_flight_information(flight_number):
    api_key = 'dd57d0d1c2cc386972d8ec3c6c7826ca'
    url = f'http://api.aviationstack.com/v1/flights?access_key={api_key}&flight_iata={flight_number}'
    response = requests.get(url)
    data = response.json()
    return data

# Define callback function to update flight information based on user input
@app.callback(
    Output('flight-output', 'children'),
    [Input('get-flight-button', 'n_clicks')],
    [dash.dependencies.State('flight-input', 'value')]
)
def update_flight_information(n_clicks, flight_number):
    if n_clicks > 0:
        flight_data = get_flight_information(flight_number)
        if 'error' in flight_data:
            return f"Error: {flight_data['error']['info']}"
        else:
            flight = flight_data['data'][0]
            departure = flight['departure']
            arrival = flight['arrival']
            airline = flight['airline']
            flight_number = flight['flight']['number']
            flight_status = flight['flight_status']

            return html.Div([
                html.H2(f"Flight Information for {airline['name']} {flight_number}"),
                html.P(f"Flight Status: {flight_status}"),
                html.H3("Departure"),
                html.P(f"Airport: {departure['airport']} ({departure['iata']})"),
                html.P(f"Terminal: {departure['terminal']}, Gate: {departure['gate']}"),
                html.P(f"Scheduled Departure: {departure['scheduled']}"),
                html.P(f"Estimated Departure: {departure['estimated']}"),
                html.H3("Arrival"),
                html.P(f"Airport: {arrival['airport']} ({arrival['iata']})"),
                html.P(f"Terminal: {arrival['terminal']}, Gate: {arrival['gate']}, Baggage: {arrival['baggage']}"),
                html.P(f"Scheduled Arrival: {arrival['scheduled']}"),
                html.P(f"Estimated Arrival: {arrival['estimated']}")
            ])

# Run the Dash application
if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>